In [1]:
import geopandas as gpd
import rasterio
import pandas as pd
import numpy as np
import rasterio
from skimage.feature import graycomatrix, graycoprops
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings("ignore") 
warnings.filterwarnings("ignore", category=UserWarning)

### Load sample points and sar data


In [ ]:
# ## palisades
shp_path='sample/palisades_random_sample.shp'
# # # shp_path='../../sample_collection/palisades_random_sample_polygon.shp'
is_polygon=False
gdf = gpd.read_file(shp_path)


glcm_raster_path="input/palisades_glcm_pca_ready.tif"
input_sar_image_path  = "input/palisades_s1_ready.tif"

thermal_image_path="input/palisades_thermal_ready.tif"
dnbr_image_path="input/palisades_dnbr_ready.tif"


final_columns_names=["dTRAD","dNBR","RBD_VV", "RBD_VH", "RBR_VV", "RBR_VH","ΔRVI",'p1','p2','p3','p4'] #,'Δvv_vh_ratio',"ΔRDFI","RCBI"
output_sar_image_path="output/feature_image/palisades_sar_indices_glcm_dtrad_dnbr.tif"
output_file_path="output/sample_feature/palisades_sar_indices_glcm_dtrad_dnbr.csv"


# # # final_columns_names=["dTRAD","dNBR","RBD_VV", "RBD_VH", "RBR_VV", "RBR_VH","ΔRVI",'Δvv_vh_ratio',"ΔRDFI","RCBI"]
# # # output_sar_image_path="output/feature_image/palisades_sar_optical_indices.tif"
# # # output_file_path="output/sample_feature/palisades_sar_optical_indices.csv"


In [ ]:
# # Load training points
# shp_path = "../../sample_collection/eaton_random_sample.shp"
# gdf = gpd.read_file(shp_path)

# input_sar_image_path  = "input/eaton_s1_ready.tif"
# glcm_raster_path="input/eaton_glcm_pca_ready.tif"
# thermal_image_path="input/eaton_thermal_ready.tif"
# dnbr_image_path="input/eaton_dnbr_ready.tif"
# #following are the input for the model selection and training

# final_columns_names=["dTRAD","dNBR","RBD_VV", "RBD_VH", "RBR_VV", "RBR_VH","ΔRVI",'p1','p2','p3','p4']
# output_file_path="output/sample_feature/eaton_sar_indices_glcm_dtrad_dnbr.csv"
# output_sar_image_path="output/feature_image/eaton_sar_indices_glcm_dtrad_dnbr.tif"

# # final_columns_names=["dTRAD","dNBR","RBD_VV", "RBD_VH", "RBR_VV", "RBR_VH","ΔRVI",'Δvv_vh_ratio',"ΔRDFI","RCBI"]
# # output_file_path="output/sample_feature/eaton_sar_optical_indices.csv"
# # output_sar_image_path="output/feature_image/eaton_sar_optical_indices.tif"

# # final_columns_names=["vh_post","vv_post","vh_pre","vv_pre"]
# # "dTRAD","dNBR","dod_RBR_VH",dod_RBR_VV
# # RCBI: https://www.sciencedirect.com/science/article/abs/pii/S0924271619302242
# # final_columns_names=['t1','t2','t3','t4','t5','t6','t7','t8','t9','t10','t11','t12','t13','t14','t15','t16','t17','t18','t19','t20']


In [4]:
add_bands=False
add_indices=True
add_glcm=True
thermal_band=True
add_dnbr=True

In [5]:
"""
Log10 of negative value is 0, thus applicanle in calculating the differnce
"""
from scipy.ndimage import generic_filter

# Open input raster
with rasterio.open(input_sar_image_path) as src:
    profile = src.profile  # Get metadata
    bands = src.read()  # Read all bands

profile.update(count=len(final_columns_names))
print(len(final_columns_names),"len(final_columns_names)")
epsilon = 1e-6
band_index=0
with rasterio.open(output_sar_image_path, "w", **profile) as dst:
    if add_bands:
        for i in range(len(bands)):
            dst.write(bands[i], i + 1)
            dst.set_band_description(i + 1, final_columns_names[i])  # Assign names
            band_index=i+1

    
    if thermal_band:
        with rasterio.open(thermal_image_path) as TRAD_dataset:
            
            # profile = src.profile  # Get metadata
            trad_band = TRAD_dataset.read()
            band_index=band_index+1
            dst.write(trad_band[0], band_index)
            dst.set_band_description(band_index, "dTRAD")
    
    if add_dnbr:
        with rasterio.open(dnbr_image_path) as dnbr_dataset:
            # profile = src.profile  # Get metadata
            dnbr_band = dnbr_dataset.read()
            band_index=band_index+1
            dst.write(dnbr_band[0], band_index)
            dst.set_band_description(band_index, "dNBR")
    
    if add_indices:
        # vh_post,vv_post,vh_pre, vv_pre=bands[0],bands[1],bands[2],bands[3]
        vh_pre,vv_pre,vh_post, vv_post=bands[0],bands[1],bands[2],bands[3]
        vv_post = np.nan_to_num(vv_post, nan=0)
        vh_post = np.nan_to_num(vh_post, nan=0)
        vv_pre = np.nan_to_num(vv_pre, nan=0)
        vh_pre = np.nan_to_num(vh_pre, nan=0)

        vv_post = np.where(vv_post <= 0, epsilon, vv_post)
        vh_post = np.where(vh_post <= 0, epsilon, vh_post)
        vv_pre = np.where(vv_pre <= 0, epsilon, vv_pre)
        vh_pre = np.where(vh_pre <= 0, epsilon, vh_pre)

        # vh_post,vv_post,vh_pre, vv_pre=(bands[2]+bands[6])/2,(bands[3]+bands[7])/2,(bands[0]+bands[4])/2,(bands[1]+bands[5])/2

        # print(np.min(vh_post),np.max(vh_post))
        band_index=band_index+1
        RBD_VV_band = vv_post - vv_pre
        # X_min,X_max = np.min(RBD_VV_band),np.max(RBD_VV_band)
        # RBD_VV_band = (RBD_VV_band - X_min) / (X_max - X_min)
        dst.write(RBD_VV_band, band_index)
        dst.set_band_description(band_index, "RBD_VV")


        band_index=band_index+1
        RBD_VH_band = vh_post - vh_pre
        # X_min,X_max = np.min(RBD_VH_band),np.max(RBD_VH_band)
        # RBD_VH_band = (RBD_VH_band - X_min) / (X_max - X_min)
        dst.write(RBD_VH_band, band_index)
        dst.set_band_description(band_index, "RBD_VH")

        band_index=band_index+1
        # RBR_VV_band = vv_post/ (vv_pre + epsilon)
        RBR_VV_band = (np.log10(vv_post))-(np.log10(vv_pre)) #logarithmic ratio
        # RBR_VV_band=np.log(RBR_VV_band + epsilon) 
        # X_min,X_max = np.min(RBR_VV_band),np.max(RBR_VV_band)
        # RBR_VV_band = (RBR_VV_band - X_min) / (X_max - X_min)
        dst.write(RBR_VV_band, band_index)
        dst.set_band_description(band_index, "RBR_VV")
        
        # band_index=band_index+1
        # # pre_std=np.std(vv_pre) #global
        # window_size = 7
        # # Apply local standard deviation filter
        # pre_std_local = generic_filter(vv_pre, np.std, size=window_size)
        # RBR_VV_band=(np.log10(vv_post))-(np.log10(vv_pre)) #logarithmic ratio
        # dod_RBR_VV= np.abs(RBR_VV_band)/pre_std_local
        # dst.write(dod_RBR_VV, band_index)
        # dst.set_band_description(band_index, "dod_RBR_VV")


        band_index=band_index+1
        # RBR_VH_band = vh_post / (vh_pre+epsilon)
        RBR_VH_band = (np.log10(vh_post)) - (np.log10(vh_pre))
        # RBR_VH_band=np.log(RBR_VH_band + epsilon) 
        # X_min,X_max = np.min(RBR_VH_band),np.max(RBR_VH_band)
        # RBR_VH_band = (RBR_VH_band - X_min) / (X_max - X_min)
        dst.write(RBR_VH_band, band_index)
        dst.set_band_description(band_index, "RBR_VH")

        # band_index=band_index+1
        # # pre_std=np.std(vh_pre)
        # window_size = 7
        # # Apply local standard deviation filter
        # pre_std_local = generic_filter(vv_pre, np.std, size=window_size)
        # RBR_VH_band=(np.log10(vh_post))-(np.log10(vh_pre))
        # dod_RBR_VH= np.abs(RBR_VH_band)/pre_std_local
        # dst.write(dod_RBR_VH, band_index)
        # dst.set_band_description(band_index, "dod_RBR_VH")

        band_index=band_index+1
        # RVI_post = 4* vh_post/(vv_post+vh_post+epsilon) #4* vh/(vv+vh)
        # RVI_pre = 4* vh_pre/(vh_pre+vv_pre+epsilon) #4* vh/(vv+vh)
        RVI_post = (10*np.log10((4* vh_post) ))-(10*np.log10(vv_post+vh_post)) #4* vh/(vv+vh)
        RVI_pre = (10*np.log10((4* vh_pre)))-(10*np.log10(vh_pre+vv_pre)) #4* vh/(vv+vh)
        delta_RVI = RVI_post  - RVI_pre
        # delta_RVI=10*np.log10(delta_RVI+epsilon)
        # delta_RVI = np.log(RVI_post + epsilon) - np.log(RVI_pre + epsilon)
        # X_min, X_max = np.min(delta_RVI), np.max(delta_RVI)
        # delta_RVI = (delta_RVI - X_min) / (X_max - X_min)
        dst.write(delta_RVI, band_index)
        dst.set_band_description(band_index, "ΔRVI")

        # band_index=band_index+1
        # # vv_vh_ratio_post = vv_post/(vh_post+epsilon) 
        # # vv_vh_ratio_pre = vv_pre/(vh_pre+epsilon)
        # vv_vh_ratio_post = (10*np.log10(vv_post))-(10*np.log10(vh_post))
        # vv_vh_ratio_pre = (10*np.log10(vv_pre))-(10*np.log10(vh_pre))
        # delta_vv_vh_ratio=vv_vh_ratio_post-vv_vh_ratio_pre
        # # delta_vv_vh_ratio=10*np.log10(delta_vv_vh_ratio+epsilon)
        # # X_min, X_max = np.min(delta_vv_vh_ratio), np.max(delta_vv_vh_ratio)
        # # delta_vv_vh_ratio = (delta_vv_vh_ratio - X_min) / (X_max - X_min)
        # # delta_vv_vh_ratio=np.log(vv_vh_ratio_post+epsilon)-np.log(vv_vh_ratio_pre+epsilon)
        # dst.write(delta_vv_vh_ratio, band_index)
        # dst.set_band_description(band_index, "Δ_vv_vh_ratio")

        # band_index=band_index+1
        # RFDI_pre = (vv_pre - vh_pre) / (vv_pre + vh_pre)
        # RFDI_post = (vv_post - vh_post) / (vv_post + vh_post)
        # delta_RFDI = RFDI_post - RFDI_pre
        # # X_min, X_max = np.min(delta_RFDI), np.max(delta_RFDI)
        # # delta_RFDI = (delta_RFDI - X_min) / (X_max - X_min)
        # dst.write(delta_RFDI, band_index)
        # dst.set_band_description(band_index, "ΔRDFI")

        # band_index=band_index+1
        # #Radar convolution burn index
        # RCBI=(vv_post-vv_pre)+(vh_post-vh_pre)
        # dst.write(RCBI, band_index)
        # dst.set_band_description(band_index, "RCBI")
    
    if add_glcm:
        band_index=band_index+1
        with rasterio.open(glcm_raster_path) as glcm_dst:
            glcm_bands = glcm_dst.read()  
            print(len(glcm_bands))
            for i in range(int(len(glcm_bands))):
            # for i in range(7,8):
            
                dst.write(glcm_bands[i], band_index)
                band_index=band_index+1

        # # glcm_bands_counts=len(glcm_bands)/2

        # band_index=band_index+1
        # with rasterio.open(glcm_raster_path) as glcm_dataset:
        #     glcm_bands = glcm_dataset.read()  # Read all bands
        #     # for i in range(16,20):
        #     #     difference=glcm_bands[i]-glcm_bands[i+20]
        #     #     dst.write(difference, band_index)
        #     #     band_index=band_index+1
        #     #     break
        #     for i in range(int(len(glcm_bands)/2)):
        #         corresponding_index,index,  = i, int(len(bands)/2)+i
        #         difference=glcm_bands[corresponding_index]-glcm_bands[index]
        #         X_min, X_max = np.min(difference), np.max(difference)
        #         print(X_max, X_min)
        #         difference = (difference - X_min) / (X_max - X_min)
        #         dst.write(difference+epsilon, band_index)
        #         band_index=band_index+1
    

11 len(final_columns_names)
4


In [6]:
if not is_polygon:
    # dataset=rasterio.open()
    with rasterio.open(output_sar_image_path) as dataset:
        print(dataset.count)
        # Get coordinates of training points
        coords = [(x, y) for x, y in zip(gdf.geometry.x, gdf.geometry.y)]

        # Extract values from all bands
        values = [v for v in dataset.sample(coords)]

        # Convert to DataFrame with proper band names
        df = pd.DataFrame(values, columns=final_columns_names)

        # Merge with training sample points
        gdf = gdf.reset_index(drop=True)
        final_df = pd.concat([gdf, df], axis=1)

        # Save as CSV for ML training
        final_df.to_csv(output_file_path, index=False)




##if polygon---------------------------------------

if is_polygon:
    from rasterio.mask import mask
    # Storage for final DataFrame
    all_pixel_data = []

    with rasterio.open(output_sar_image_path) as src:
        for idx, row in gdf.iterrows():
            polygon_geom = [row.geometry.__geo_interface__]
            class_label = row['class']  # Make sure 'class' exists in your shapefile

            try:
                # Mask raster by the polygon shape, crop to polygon bounds
                out_image, out_transform = mask(src, polygon_geom, crop=True)
            except ValueError:
                # Skip polygons that do not intersect raster
                continue

            # Identify valid pixels (not nodata)
            mask_band = out_image[0] != src.nodata
            rows, cols = np.where(mask_band)

            if len(rows) == 0:
                continue  # Skip empty areas

            # Get centroid coordinates of the valid pixels
            xs, ys = rasterio.transform.xy(out_transform, rows, cols, offset='center')
            coords = list(zip(xs, ys))

            # Extract pixel values at those positions
            pixel_values = out_image[:, rows, cols].transpose()

            # Build DataFrame for this polygon
            df_coords = pd.DataFrame(coords, columns=["x", "y"])
            df_values = pd.DataFrame(pixel_values, columns=final_columns_names)
            df_class = pd.DataFrame({'class': [class_label] * len(df_coords)})

            df_full = pd.concat([df_coords, df_values, df_class], axis=1)
            all_pixel_data.append(df_full)

    # Combine all into one DataFrame
    final_df = pd.concat(all_pixel_data, ignore_index=True)

    # Save to CSV
    final_df.to_csv(output_file_path, index=False)
    print(f"Saved with class labels: {output_file_path}")



11


In [7]:
final_df.head(10)

,id,Dnbr1,dTRAD1,class,geometry,dTRAD,dNBR,RBD_VV,RBD_VH,RBR_VV,RBR_VH,ΔRVI,p1,p2,p3,p4
0,0.0,-0.012932,0.390035,0,POINT (402563.977 3781947.786),-264.0,0.015814,-0.001532,-0.000140,-0.017192,-0.004792,0.093313,15.480120,-0.926525,0.071767,1.093092
1,1.0,0.257881,0.329700,1,POINT (401001.224 3788317.096),-426.0,0.277569,0.000668,0.006971,0.002718,0.116885,0.918899,7.130439,-1.373819,-0.138479,1.178800
2,2.0,-0.018341,0.293110,0,POINT (396536.715 3779106.757),-509.0,0.004437,-0.139107,0.019352,-0.272681,0.188135,3.809484,-1.066822,0.227925,-0.628804,1.603273
3,3.0,0.045327,0.274815,0,POINT (394725.939 3780181.137),-560.0,0.027886,-0.031607,0.010836,-0.006526,0.019814,0.236676,-57.975876,10.718716,14.116808,-2.085377
4,4.0,0.015972,0.245232,0,POINT (403621.816 3785897.184),-627.0,0.020026,-0.000094,0.000823,-0.001096,0.046441,0.393968,18.265457,1.545770,0.134699,-1.690913
5,5.0,0.004455,0.215648,0,POINT (391215.942 3788010.582),-702.0,0.003407,0.034805,0.026041,0.179420,0.300533,0.840150,6.280194,-8.442175,0.797638,0.481441
6,6.0,0.189714,0.562086,1,POINT (396739.847 3785803.935),116.0,0.184698,0.054223,0.000846,0.122891,0.006107,-0.887743,-20.630966,1.023282,-1.293941,1.545691
7,7.0,0.355512,0.508758,1,POINT (396734.55 3787246.972),111.0,0.369515,-0.001912,-0.020591,-0.009951,-0.333456,-2.455688,3.264849,1.236653,-0.500378,0.367017
8,8.0,0.022919,0.337096,0,POINT (400108.795 3779769.036),-404.0,0.013089,0.099466,-0.000539,0.314352,-0.008157,-2.650535,4.467752,0.694199,-0.439267,0.790081
9,9.0,0.018437,0.305566,0,POINT (392028.617 3779193.377),-472.0,0.008507,0.000356,-0.004352,0.002013,-0.112853,-0.943311,11.601834,-1.191680,-0.087813,1.787176


In [8]:
df.agg(['min', 'max'])

,dTRAD,dNBR,RBD_VV,RBD_VH,RBR_VV,RBR_VH,ΔRVI,p1,p2,p3,p4
min,-1095.0,-0.176160,-9.457702,-1.275994,-1.489750,-1.040811,-13.422253,-79.920319,-41.933708,-39.764244,-18.804724
max,1206.0,0.767681,7.696883,1.070527,1.591699,1.017669,14.103199,19.954237,38.019337,68.390045,8.418229


In [9]:
# 	dTRAD	dNBR	RBD_VV	RBD_VH	RBR_VV	dod_RBR_VV	RBR_VH	dod_RBR_VH	ΔRVI	Δvv_vh_ratio	ΔRDFI	RCBI
# min	0.057610	-0.248237	-1.953554	-0.422484	-0.756728	0.000416	-1.267929	0.000264	-14.016992	-9.988633	-0.875591	-2.064492
# max	0.958609	0.739910	5.741815	0.638357	1.805561	216.663483	1.513628	185.473175	7.720948	15.615835	1.214909	5.602402